L'objectif de ce code est de pouvoir générer un JSON que je peux push sur le back via une API Flask afin de générer un workflow d'un processus métier respectant le langage BPMN.
Ce Workflow sera défini dans un JSON complet permettant au front d'afficher le BPMN.

Ce JSON aura en entré :
- Un nom de processus
- Une description textuelle
- Des KPI a analyser
- Différents documents que l'on peut analyser dans le processus

Ces différentes étapes seront injecter dans un JSON qui sera lui même transmis à un chatbot qui génrerera un workflow BPMN sous la forme du json suivant : 

```json
{
  "process": {
    "name": "Processus de gestion des commandes",
    "description": "Ce processus permet de gérer les commandes clients",
    "Etape_Workflow : [{
      "name": "Recupération des documents",
      "description": "Description de l'étape 1",
      "Type": "INPUT"  
        },
        {
      "name": "Recupération des documents",
      "description": "Description de l'étape 1",
      "Type": "XOR"  
        },
        {....}
      }]
    }
}
```

On effectuera une visualisation en préalable sur python du diagramme BPMN afin de comprendre au mieux le rendu et voir les différents liens qui sont fait

In [1]:
import pandas as pd
import json
import requests
import os
import flask

In [2]:
def generate_bpmn(data):
    """
    Génère un JSON BPMN à partir des données fournies.
    
    :param data: Dictionnaire contenant les informations du processus.
    :return: JSON structuré du processus BPMN.
    """
    process_name = data.get('name')
    process_description = data.get('description')
    kpis = data.get('kpis', [])
    documents = data.get('documents', [])
    steps = data.get('steps', [])

    # Ajout de champs supplémentaires pour chaque étape
    for i, step in enumerate(steps):
        step["nextStep"] = steps[i + 1]["name"] if i + 1 < len(steps) else None
        step["roles"] = step.get("roles", [])

    # Construction du JSON BPMN
    bpmn_json = {
        "process": {
            "name": process_name,
            "description": process_description,
            "KPI": kpis,
            "Documents": documents,
            "Etape_Workflow": steps
        }
    }

    return bpmn_json


# Exemple de données de test
test_data = {
    "name": "Processus de gestion des commandes",
    "description": "Ce processus permet de gérer les commandes clients",
    "kpis": ["KPI1", "KPI2"],
    "documents": ["Doc1", "Doc2"],
    "steps": [
        {
            "name": "Recupération des documents",
            "description": "Description de l'étape 1",
            "Type": "INPUT",
            "roles": ["Admin"]
        },
        {
            "name": "Vérification des documents",
            "description": "Description de l'étape 2",
            "Type": "XOR",
            "roles": ["Manager"]
        },
        {
            "name": "Validation finale",
            "description": "Description de l'étape 3",
            "Type": "END",
            "roles": ["Supervisor"]
        }
    ]
}

# Test de la fonction
result = generate_bpmn(test_data)
print("JSON BPMN généré :")
import json
print(json.dumps(result, indent=4, ensure_ascii=False))

JSON BPMN généré :
{
    "process": {
        "name": "Processus de gestion des commandes",
        "description": "Ce processus permet de gérer les commandes clients",
        "KPI": [
            "KPI1",
            "KPI2"
        ],
        "Documents": [
            "Doc1",
            "Doc2"
        ],
        "Etape_Workflow": [
            {
                "name": "Recupération des documents",
                "description": "Description de l'étape 1",
                "Type": "INPUT",
                "roles": [
                    "Admin"
                ],
                "nextStep": "Vérification des documents"
            },
            {
                "name": "Vérification des documents",
                "description": "Description de l'étape 2",
                "Type": "XOR",
                "roles": [
                    "Manager"
                ],
                "nextStep": "Validation finale"
            },
            {
                "name": "Validation finale",
 

In [3]:
import pydotplus
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def visualize_bpmn(diagram, output_file_path='diagramme_bpmn.png'):
    """
    Génère une visualisation du diagramme BPMN et l'enregistre en tant qu'image PNG.

    :param diagram: Instance de BpmnDiagramGraph.
    :param output_file_path: Chemin du fichier de sortie pour l'image PNG.
    """
    g = diagram.diagram_graph
    graph = pydotplus.Dot(graph_type='digraph')

    # Ajouter les nœuds
    for node in g.nodes(data=True):
        node_id = node[0]
        node_data = node[1]
        node_name = node_data.get('name', '')
        node_type = node_data.get('type', '')

        if node_type == 'task':
            shape = 'box'
            style = 'rounded'
        elif node_type == 'exclusive_gateway':
            shape = 'diamond'
            style = ''
        else:
            shape = 'ellipse'
            style = ''

        graph_node = pydotplus.Node(name=node_id, label=node_name, shape=shape, style=style)
        graph.add_node(graph_node)

    # Ajouter les arêtes
    for edge in g.edges(data=True):
        source = edge[0]
        target = edge[1]
        edge_data = edge[2]
        edge_label = edge_data.get('name', '')
        graph_edge = pydotplus.Edge(src=source, dst=target, label=edge_label)
        graph.add_edge(graph_edge)

    # Générer le fichier PNG
    graph.write_png(output_file_path)

    # Afficher l'image
    img = mpimg.imread(output_file_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [1]:
import json
from graphviz import Digraph

def generate_bpmn_from_json(json_file, output_file='processus_bpmn'):
    # Charger les données JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Initialiser le graphe Graphviz
    bpmn_graph = Digraph(name=data['process']['name'], format='png')
    bpmn_graph.attr(rankdir='LR', size='8,5')

    # Ajouter les éléments au graphe
    elements = {}
    for step in data['process']['Etape_Workflow']:
        step_name = step['name']
        step_type = step['Type']

        # Choisir le style selon le type d'étape
        if step_type == 'INPUT':
            bpmn_graph.node(step_name, shape='ellipse', style='filled', color='lightblue')
        elif step_type == 'XOR':
            bpmn_graph.node(step_name, shape='diamond', style='filled', color='orange')
        elif step_type == 'END':
            bpmn_graph.node(step_name, shape='ellipse', style='filled', color='lightgreen')
        else:
            bpmn_graph.node(step_name, shape='box', style='rounded,filled', color='lightgrey')

        elements[step_name] = step_name

    # Ajouter les flux de séquence
    for step in data['process']['Etape_Workflow']:
        next_step = step.get('nextStep')
        if next_step and next_step in elements:
            bpmn_graph.edge(step['name'], next_step)

    # Générer le fichier de sortie
    bpmn_graph.render(output_file, cleanup=True)
    print(f"Diagramme BPMN généré : {output_file}.png")

# Exemple d'utilisation
generate_bpmn_from_json('processus.json')

Diagramme BPMN généré : processus_bpmn.png
